In [16]:
# Check for identifier
word = input("Enter a word")

import re

if re.fullmatch("[A-Za-z_]\w{0,31}", word): 
    print("Indentifier")
else:
    print("Not Identifier")

Enter a word34f
Not Identifier


In [38]:
# check if a string belongs to the RE
word = input("Enter a word")

import re

if re.fullmatch("a*((aa)|b)*", word) or re.fullmatch("a*b+"): 
    print("String is recognized")
else:
    print("String is not recognized")

Enter a worda
String is recognized


In [71]:
def commonprefix(derives):
    t = min(derives)
    for s in derives:
        for i, c in enumerate(s):
            if c != t[i]:
                return t[:i]
    return t

production = input("Enter a production rule")
left = production[0]
derives = production[production.index('>') + 1:].split('|')

prefix = commonprefix(derives)

if (len(prefix) == 0 or len(derives) == 1):
    print(left + "->" + "|".join(derives))
else:
    derives = ["Z" + d[len(prefix):] for d in derives]
    print(left + "->" + "|".join(derives))
    print("Z->" + prefix)

Enter a production rulea->b|c
a->b|c


In [2]:
from collections import defaultdict

def removeLeftRecursion(production):
    a = production[0]
    derives = production[production.index(">") + 1:].split("|")
    betas = []
    alphas = []
    for each in derives:
        if each[0] != a:
            betas.append(each)
        else:
            alphas.append(each[1:])
    if len(betas) == len(derives):
        return [production]
    global n
    ad = extra[n]
    n += 1
    s = a + "->" + "|".join((i + ad) for i in betas)
    s2 = ad + "->" + "|".join((i + ad) for i in alphas) + "|e"
    return [s, s2]

#cfg = [item for item in input("Enter the list items : ").split()]
cfg = [item for item in "E->E+T|T T->TF|F F->F*|a|b".split()]
start = 'E'

isPresent = defaultdict(lambda: False)
for production in cfg:
    isPresent[production[0]] = True
extra = []
n = 0
for c in range(ord('A'), ord('Z') + 1):
    if isPresent[chr(c)] == False:
        extra.append(chr(c))
ans = []
        
for production in cfg:
    for each in removeLeftRecursion(production): 
        ans.append(each)

cfg = ans
print(cfg)

['E->TA', 'A->+TA|e', 'T->FB', 'B->FB|e', 'F->aC|bC', 'C->*C|e']


In [3]:
def isTerminal(c):
    return not (ord(c) >= ord('A') and ord(c) <= ord('Z'))

def getFirst(c):
    if vis[c]:
        return first[c]
    if isTerminal(c):
        vis[c] = True
        first[c].add(c)
        return first[c]
    
    for each in derives[c]:
            ind = 0
            while(ind < len(each)):
                f = 0
                for x in getFirst(each[ind]):
                    if x == 'e':
                        f = 1
                    else:
                        first[c].add(x)
                if f == 0:
                    break
                ind += 1
            if ind == len(each) and f == 1:
                first[c].add('e')
    vis[c] = True
    return first[c]

first = defaultdict(lambda: set())
vis = defaultdict(lambda: False)

derives = dict()
for production in cfg:
    derives[production[0]] = production[3:].split('|')
    
terminals = []

for production in cfg:
    for x in production:
        if isTerminal(x):
            vis[x] = True
            first[x].add(x)
            terminals.append(x)
    getFirst(production[0])
    
for production in cfg:
    print(production[0], first[production[0]])

E {'b', 'a'}
A {'+', 'e'}
T {'b', 'a'}
B {'b', 'e', 'a'}
F {'b', 'a'}
C {'*', 'e'}


In [4]:
follow = defaultdict(lambda: set())
follow[start].add('$')
vis = defaultdict(lambda: False)

def getFollow(c):
    if vis[c]:
        return follow[c]
    vis[c] = True
    for production in cfg:
        for d in derives[production[0]]:
            for ind in range(len(d)):
                if d[ind] != c:
                    continue
                ind += 1
                f = 1
                while(ind < len(d)):
                    f = 0
                    for x in first[d[ind]]:
                        if x != 'e':
                            follow[c].add(x)
                        else:
                            f = 1
                    if f == 0:
                        break   
                    ind += 1
                if ind >= len(d) and f == 1:
                    for x in getFollow(production[0]):
                        follow[c].add(x)
    return follow[c]
            
for production in cfg:
    getFollow(production[0])
    print(production[0], follow[production[0]])

E {'$'}
A {'$'}
T {'+', '$'}
B {'+', '$'}
F {'$', 'b', '+', 'a'}
C {'a', 'b', '+', '$'}


In [5]:
table = defaultdict(lambda: '')

def fillTable(c):
    for each in derives[c]:
            ind = 0
            while(ind < len(each)):
                f = 0
                for x in first[each[ind]]:
                    if x == 'e':
                        f = 1
                    else:
                        table[c + x] = each
                if f == 0:
                    break
                ind += 1
            if ind == len(each) and f == 1:
                for x in follow[c]:
                    table[c + x] = each

for production in cfg:
    fillTable(production[0])

print(table)

defaultdict(<function <lambda> at 0x0000016B66C3CB80>, {'Eb': 'TA', 'Ea': 'TA', 'A+': '+TA', 'A$': 'e', 'Tb': 'FB', 'Ta': 'FB', 'Bb': 'FB', 'Ba': 'FB', 'B+': 'e', 'B$': 'e', 'Fa': 'aC', 'Fb': 'bC', 'C*': '*C', 'Ca': 'e', 'Cb': 'e', 'C+': 'e', 'C$': 'e'})


In [6]:
#s = input("Enter the string")
s = "a+b*a$"
i = 0
f = 0
stack = ['$', start]

while(i < len(s) and len(stack) > 0):
    ch = stack.pop()
    if isTerminal(ch):
        if ch == s[i]:
            i = i + 1
        elif ch != 'e':
            print('String is not accepted')
            f = 1
            break
    else:
        t = table[ch + s[i]][::-1]
        if len(t) == 0:
            print('String is not accepted')
            f = 1
            break
        stack.extend(t)
        print(ch, s[i], stack)

if f == 0:
    print('String is accepted')

E a ['$', 'A', 'T']
T a ['$', 'A', 'B', 'F']
F a ['$', 'A', 'B', 'C', 'a']
C + ['$', 'A', 'B', 'e']
B + ['$', 'A', 'e']
A + ['$', 'A', 'T', '+']
T b ['$', 'A', 'B', 'F']
F b ['$', 'A', 'B', 'C', 'b']
C * ['$', 'A', 'B', 'C', '*']
C a ['$', 'A', 'B', 'e']
B a ['$', 'A', 'B', 'F']
F a ['$', 'A', 'B', 'C', 'a']
C $ ['$', 'A', 'B', 'e']
B $ ['$', 'A', 'e']
A $ ['$', 'e']
String is accepted


In [7]:
# RECURSIVE DESCENT PARSING

#cfg = [item for item in input("Enter the list items : ").split()]
cfg = [item for item in "E->Ab|b A->ab".split()]
start = 'E'
ans = []
n = 0
for production in cfg:
    for each in removeLeftRecursion(production): 
        ans.append(each)
cfg = ans
#str = input("Enter the string")
str = "ab"
ind = 0

derives = dict()
checkNext = dict()
for production in cfg:
    derives[production[0]] = production[3:].split('|')
    checkNext[production[0]] = 0
    
    
def isAccepted(c):
    global ind
    if isTerminal(c):
        if c == 'e':
            return True
        if c == str[ind]: 
            ind += 1
            return True
        return False

    prevInd = ind
    while checkNext[c] != len(derives[c]):
        ind = prevInd
        t = derives[c][checkNext[c]]
        checkNext[c] = checkNext[c] + 1
        f = 0
        for i in t:
            temp = isAccepted(i)
            if not temp:
                f = 1
                break
            elif ind == len(str):
                return True
        if f == 0:
            if c == start:
                return ind == len(str)
            return True
    return False

isAccepted(start)

True